In [1]:
!nvidia-smi

Sat Aug 10 11:49:32 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   56C    P8              10W /  70W |      0MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [2]:
!pip install ultralytics

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.3/41.3 kB 3.4 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-manylinux2014_x86_64.

In [10]:
import os
import csv
import cv2
from ultralytics import YOLO, solutions
from IPython.display import display, Image
from IPython import display
display.clear_output()

In [8]:
# Load the trained model
model = YOLO('./staff_tracker/train_final/weights/best.pt')
video_path = './data/sample.mp4'
video_name = os.path.splitext(os.path.basename(video_path))[0]

# # Run inference on the source
results = model.track(source=video_path, show=False, save=True,
                      tracker='botsort.yaml', project='staff_tracker')



WARNING  inference results will accumulate in RAM unless `stream=True` is passed, causing potential out-of-memory
errors for large sources or long-running streams and videos. See https://docs.ultralytics.com/modes/predict/ for help.

Example:
    results = model(source=..., stream=True)  # generator of Results objects
    for r in results:
        boxes = r.boxes  # Boxes object for bbox outputs
        masks = r.masks  # Masks object for segment masks outputs
        probs = r.probs  # Class probabilities for classification outputs

video 1/1 (frame 1/1341) c:\Users\User\OneDrive - sjtu.edu.cn\SJTU\Road to Machine Learning Engineer\Job Tech Assessment\footfallcam\data\sample.mp4: 480x640 (no detections), 69.8ms
video 1/1 (frame 2/1341) c:\Users\User\OneDrive - sjtu.edu.cn\SJTU\Road to Machine Learning Engineer\Job Tech Assessment\footfallcam\data\sample.mp4: 480x640 (no detections), 16.5ms
video 1/1 (frame 3/1341) c:\Users\User\OneDrive - sjtu.edu.cn\SJTU\Road to Machine Learning En

In [11]:
data = []

cap = cv2.VideoCapture(f'./staff_tracker/track/{video_name}.avi')
assert cap.isOpened(), "Error reading video file"
w, h, fps = (int(cap.get(x)) for x in (cv2.CAP_PROP_FRAME_WIDTH, cv2.CAP_PROP_FRAME_HEIGHT, cv2.CAP_PROP_FPS))

# Video writer
video_writer = cv2.VideoWriter(f"./staff_tracker/track/{video_name}_xy.avi", cv2.VideoWriter_fourcc(*"mp4v"), fps, (w, h))

def get_bbox_xy(frame_idx, results=results):
  frame = results[frame_idx]
  center_x, center_y = None, None
  bbox_xy = []

  for bbox in frame.boxes:
    center_x, center_y, _, _ = bbox.xywh[0]
    center_x, center_y = int(center_x.round().item()), int(center_y.round().item())
    bbox_xy.append([center_x, center_y])

  return bbox_xy

frame_idx = 0
while cap.isOpened():
  success, frame = cap.read()
  if not success:
    break

  bbox_xy = get_bbox_xy(frame_idx, results)

  if bbox_xy:
    for bbox in bbox_xy:
      cv2.circle(frame, (bbox[0], bbox[1]), 5, (0, 0, 255), -1)
      cv2.putText(frame, f'({bbox[0]}, {bbox[1]})', (bbox[0] + 10, bbox[1] - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 2)

    data.append({'frame_idx': frame_idx, 'bbox_xy': bbox_xy})

  video_writer.write(frame)
  frame_idx += 1

# Write the data to the CSV file
with open(f'./staff_tracker/track/{video_name}.csv', mode='w', newline='') as file:
  writer = csv.writer(file)
  # Write the header
  writer.writerow(['frame_idx', 'bbox_xy'])
  # Write the data
  for row in data:
    writer.writerow([row['frame_idx'], row['bbox_xy']])

cap.release()
video_writer.release()
cv2.destroyAllWindows()

error: OpenCV(4.10.0) D:\a\opencv-python\opencv-python\opencv\modules\highgui\src\window.cpp:1295: error: (-2:Unspecified error) The function is not implemented. Rebuild the library with Windows, GTK+ 2.x or Cocoa support. If you are on Ubuntu or Debian, install libgtk2.0-dev and pkg-config, then re-run cmake or configure script in function 'cvDestroyAllWindows'
